## Measuring the full embryo length

The full embryo length is estimated from the major axis of the ellipse that has the same second moment as the binary image that represents the embryo.
With the exception of the VNC, the pixel values inside the embryo area are low and similar to the background pixels.
To split them, we first equalize the image histogram, followed by a low threshold method (triangle method).
After we have the binary image, we can use `skimage.measure.regionprops` to measure the embryo length.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from tifffile import imread
from skimage.exposure import equalize_hist

from snazzy_processing import full_embryo_length, utils

experiment_name = '20240611_25C'
root_dir = Path.cwd().parent
project_dir = root_dir.joinpath('data', experiment_name)

#### Visualizing the method

Compare the generated measurement with the original image.

In [ ]:
embs = sorted(project_dir.joinpath("embs").glob("*ch2.tif"), key=utils.emb_number)
i = 9

print(embs[i].stem)

img = imread(embs[i], key=range(0, 100))
img = np.average(img, axis=0)

equalized = equalize_hist(img)
bin = full_embryo_length.binarize(equalized)
full_embryo_length.view_full_embryo_length(bin, equalized)

## Measuring embryos with low background signal

When different markers are used, sometimes the signal in the 'non-VNC' area will be very low.
In these cases, pass `low_non_VNC=True` to `full_embryo_length.measure()`.
The following cell compares the measurement to the original image in this case.

In [ ]:
embs = sorted(project_dir.joinpath("embs").glob("*ch2.tif"), key=utils.emb_number)
i = 0

print(embs[i].stem)

img = imread(embs[i], key=range(1000, 1100))
img = np.average(img, axis=0)

equalized = equalize_hist(img)
bin = full_embryo_length.binarize_low_embryo_background(equalized)
full_embryo_length.view_full_embryo_length(bin, equalized)

#### Measuring embryo length

Displays the embryo length for all embryos found in the current project.

In [ ]:
embs = sorted(project_dir.joinpath("embs").glob("*ch2.tif"), key=utils.emb_number)

for emb in embs:
    print(emb.stem, full_embryo_length.measure(emb))

Compares embryo size with annotated data.

In [ ]:
pixel_size = 1.62
csv_path = project_dir.joinpath("emb_sizes", "embryo_size.csv")
try:
    annotated_data = full_embryo_length.get_annotated_data(csv_path)

    # maps annotated_id: embryo_id
    name_LUT = {i: i for i in range(3, 23)}
    embs = sorted(project_dir.joinpath("embs").glob("*ch2.tif"), key=utils.emb_number)

    measured = {
        utils.emb_number(e.stem): full_embryo_length.measure(e, low_non_VNC=True)
        for e in embs
    }

    annotated = {}
    for ann_id, size in annotated_data:
        emb_id = name_LUT.get(ann_id, None)
        if emb_id is None:
            continue
        annotated[emb_id] = size

    common_keys = list(annotated.keys() & measured.keys())

    diff = [(measured[k] - annotated[k]) / measured[k] for k in common_keys]

    fig, ax = plt.subplots()
    ax.plot(common_keys, diff, "bo")
    ax.set_xticks(common_keys)
    fig.suptitle(f"Embryo size measurement - {experiment_name}")
except FileNotFoundError:
    print("Could not find the file with annotated data. Path received:")
    print(csv_path)